In [ ]:
# Insira seu project token aqui

# MARATONA BEHIND THE CODE 2020

## DESAFIO 7 - TNT

<hr>

## Installing Libs

In [ ]:
!pip install scikit-learn --upgrade

In [ ]:
!pip install xgboost --upgrade

In [ ]:
!pip install imblearn --upgrade

In [ ]:
!pip3 install cloudant

<hr>

## Download dos conjuntos de dados em formato .csv

In [1]:
import pandas as pd
from cloudant import Cloudant

In [ ]:
u = "d5aed30c-4651-48f7-96bd-bd6030237e2a-bluemix"
p = "cfad465a453fad58b800c37b2393f2cd348e34b2451b0b5d217a7bf001b63fea"
a = "d5aed30c-4651-48f7-96bd-bd6030237e2a-bluemix"
client = Cloudant(u, p, account=a, connect=True, auto_renew=True)

In [ ]:
db = client[ "thop-iot" ]

In [ ]:
response = db.all_docs(include_docs=True)

docs = []
for r in response['rows']:
    docs.append(r['doc'])
type(docs)

In [ ]:
# Insira aqui o pandasDataFrame.
df = pd.DataFrame(data=docs)

df["LAT"] = df["LAT"].astype(float)
df["LONG"] = df["LONG"].astype(float)

df.head()

In [ ]:
df.shape

In [ ]:
df_distinct = df.drop_duplicates("row")

In [ ]:
df_distinct.shape

In [ ]:
df_training_dataset = df_distinct
df_training_dataset.tail()

In [ ]:
df_training_dataset.to_csv("dataset.csv", index=False)

Sobre o arquivo "training_dataset.csv", temos algumas informações gerais sobre os usuários do app do BanCoppel:

**Tempo**

**Estação**

**LAT**

**LONG**

**Movimentação**

**Original_473**

**Original_269**

**Zero**

**Maçã-Verde**

**Tangerina**

**Citrus**

**Açaí-Guaraná**

**Pêssego**

**TARGET**

In [2]:
df_training_dataset = pd.read_csv("dataset.csv")
df_training_dataset = df_training_dataset.sort_values("row")
df_training_dataset = df_training_dataset[["Tempo", "Estação", "LAT", "LONG", "Movimentação", "Original_473",
                                          "Original_269", "Zero", "Maçã-Verde", "Tangerina", "Citrus", "Açaí-Guaraná",
                                          "Pêssego", "TARGET"]]
df_training_dataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 17017 entries, 10716 to 10635
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Tempo         17016 non-null  object 
 1   Estação       17016 non-null  object 
 2   LAT           17016 non-null  float64
 3   LONG          17016 non-null  float64
 4   Movimentação  17016 non-null  float64
 5   Original_473  17016 non-null  float64
 6   Original_269  17016 non-null  float64
 7   Zero          17016 non-null  float64
 8   Maçã-Verde    17016 non-null  float64
 9   Tangerina     17016 non-null  float64
 10  Citrus        17016 non-null  float64
 11  Açaí-Guaraná  17016 non-null  float64
 12  Pêssego       17016 non-null  float64
 13  TARGET        17016 non-null  object 
dtypes: float64(11), object(3)
memory usage: 1.9+ MB


In [3]:
df_training_dataset.nunique()

Tempo             720
Estação            25
LAT                25
LONG               25
Movimentação    14873
Original_473       81
Original_269       65
Zero               63
Maçã-Verde         43
Tangerina          42
Citrus             43
Açaí-Guaraná       42
Pêssego            42
TARGET              2
dtype: int64

<hr>

## Detalhamento do desafio: classificação binária

Este é um desafio cujo objetivo de negócio é a segmentação dos usuários de aplicativo de um banco. Para tal, podemos utilizar duas abordagens: aprendizado de máquina supervisionado (classificação) ou não-supervisionado (clustering). Neste desafio será aplicada a classificação, pois é disponível um dataset já com "labels", ou em outras palavras, já com exemplos de dados juntamente com a variável alvo.

Na biblioteca scikit-learn temos diversos algoritmos para classificação. O participante é livre para utilizar o framework que desejar para completar esse desafio.

Neste notebook será mostrado um exeplo de uso do algoritmo "Decision Tree" para classificar parte dos estudantes em seis diferentes perfís.

# Atenção!

A coluna-alvo neste desafio é a coluna ``TARGET``

<hr>

## Pre-processando o dataset antes do treinamento

### Processando valores NaN com o SimpleImputer do sklearn

Para os valores NaN, usaremos a substituição pela constante 0 como **exemplo**.

Você pode escolher a estratégia que achar melhor para tratar os valores nulos :)

Docs: https://scikit-learn.org/stable/modules/generated/sklearn.impute.SimpleImputer.html?highlight=simpleimputer#sklearn.impute.SimpleImputer

In [4]:
from sklearn.impute import SimpleImputer
import numpy as np


impute_zeros = SimpleImputer(
    missing_values=np.nan,
    strategy='constant',
    fill_value=0,
    verbose=0,
    copy=True
)

In [5]:
# Exibindo os dados ausentes do conjunto de dados antes da primeira transformação (df)
print("Valores nulos no df_training_dataset antes da transformação SimpleImputer: \n\n{}\n".format(df_training_dataset.isnull().sum(axis = 0)))

# Aplicando a transformação ``SimpleImputer`` no conjunto de dados base
impute_zeros.fit(X=df_training_dataset)

# Reconstruindo um Pandas DataFrame com os resultados
df_training_dataset_imputed = pd.DataFrame.from_records(
    data=impute_zeros.transform(
        X=df_training_dataset
    ),
    columns=df_training_dataset.columns
)

# Eliminar linha com zero
df_training_dataset_imputed = df_training_dataset_imputed[(df_training_dataset_imputed[["Estação", "Tempo"]] != 0).all(axis=1)]

# Exibindo os dados ausentes do conjunto de dados após a primeira transformação (df)
print("Valores nulos no df_training_dataset após a transformação SimpleImputer: \n\n{}\n".format(df_training_dataset_imputed.isnull().sum(axis = 0)))

Valores nulos no df_training_dataset antes da transformação SimpleImputer: 

Tempo           1
Estação         1
LAT             1
LONG            1
Movimentação    1
Original_473    1
Original_269    1
Zero            1
Maçã-Verde      1
Tangerina       1
Citrus          1
Açaí-Guaraná    1
Pêssego         1
TARGET          1
dtype: int64

Valores nulos no df_training_dataset após a transformação SimpleImputer: 

Tempo           0
Estação         0
LAT             0
LONG            0
Movimentação    0
Original_473    0
Original_269    0
Zero            0
Maçã-Verde      0
Tangerina       0
Citrus          0
Açaí-Guaraná    0
Pêssego         0
TARGET          0
dtype: int64



### Eliminando colunas indesejadas

Vamos **demonstrar** abaixo como usar o método **DataFrame.drop()**.

Docs: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.drop.html

In [6]:
df_training_dataset_imputed.tail()

,Tempo,Estação,LAT,LONG,Movimentação,Original_473,Original_269,Zero,Maçã-Verde,Tangerina,Citrus,Açaí-Guaraná,Pêssego,TARGET
17011,2019-12-26,Hospital São Paulo,-23.5984,-46.6455,25702.0,53.0,54.0,26.0,38.0,23.0,25.0,11.0,37.0,NORMAL
17012,2019-12-27,Hospital São Paulo,-23.5984,-46.6455,26236.0,43.0,52.0,17.0,33.0,18.0,21.0,5.0,35.0,REABASTECER
17013,2019-12-28,Hospital São Paulo,-23.5984,-46.6455,27174.0,34.0,50.0,9.0,28.0,15.0,16.0,43.0,32.0,NORMAL
17014,2019-12-29,Hospital São Paulo,-23.5984,-46.6455,26877.0,22.0,47.0,65.0,27.0,11.0,16.0,39.0,30.0,NORMAL
17015,2019-12-30,Hospital São Paulo,-23.5984,-46.6455,26759.0,9.0,42.0,56.0,22.0,8.0,10.0,35.0,26.0,NORMAL


In [7]:
df_training_dataset_rmcolumns = df_training_dataset_imputed.drop(columns=['LAT', 'LONG'], inplace=False)

# Atenção!

As colunas removidas acima são apenas para fim de exemplo, você pode usar as colunas que quiser e inclusive criar novas colunas com dados que achar importantes!


### Tratamento de de variáveis categóricas

Como mencionado antes, os computadores não são bons com variáveis "categóricas" (ou strings).

Dado uma coluna com variável categórica, o que podemos realizar é a codificação dessa coluna em múltiplas colunas contendo variáveis binárias. Esse processo é chamado de "one-hot-encoding" ou "dummy encoding". Se você não é familiarizado com esses termos, você pode pesquisar mais sobre isso na internet :)

In [8]:
# Tratando variáveis categóricas com o método Pandas ``get_dummies()''
# df_training = pd.get_dummies(df_training_dataset_rmcolumns, columns=['Variável a ser aplicado método getDumies()'])
# df_training = df_training_dataset_rmcolumns
# df_training.tail()

# Atenção!

A coluna **TARGET** deve ser mantida como uma string. Você não precisa processar/codificar a variável-alvo.

<hr>

## Treinando um classificador com base em uma árvore de decisão

### Selecionando FEATURES e definindo a variável TARGET

In [9]:
df_training_dataset_rmcolumns.tail(20)

,Tempo,Estação,Movimentação,Original_473,Original_269,Zero,Maçã-Verde,Tangerina,Citrus,Açaí-Guaraná,Pêssego,TARGET
16996,2019-12-11,Hospital São Paulo,25311.0,30.0,14.0,58.0,37.0,38.0,33.0,26.0,13.0,NORMAL
16997,2019-12-12,Hospital São Paulo,25341.0,23.0,8.0,52.0,34.0,34.0,29.0,24.0,7.0,NORMAL
16998,2019-12-13,Hospital São Paulo,27771.0,16.0,65.0,47.0,34.0,30.0,26.0,21.0,2.0,NORMAL
16999,2019-12-14,Hospital São Paulo,27743.0,6.0,57.0,42.0,34.0,25.0,22.0,16.0,43.0,NORMAL
17000,2019-12-15,Hospital São Paulo,24357.0,86.0,49.0,36.0,28.0,22.0,20.0,11.0,41.0,NORMAL
17001,2019-12-16,Hospital São Paulo,27837.0,79.0,46.0,26.0,24.0,20.0,17.0,7.0,36.0,NORMAL
17002,2019-12-17,Hospital São Paulo,26853.0,69.0,38.0,22.0,22.0,18.0,13.0,2.0,31.0,NORMAL
17003,2019-12-18,Hospital São Paulo,26636.0,59.0,34.0,12.0,22.0,16.0,11.0,43.0,25.0,NORMAL
17004,2019-12-19,Hospital São Paulo,27365.0,47.0,30.0,5.0,19.0,11.0,7.0,38.0,22.0,REABASTECER
17005,2019-12-20,Hospital São Paulo,26406.0,34.0,26.0,65.0,13.0,8.0,2.0,35.0,19.0,NORMAL


In [10]:
df_training_dataset_rmcolumns["Tempo"] = pd.to_datetime(df_training_dataset_rmcolumns["Tempo"], format='%Y-%m-%d', errors='coerce')

In [11]:
df_training_dataset_rmcolumns.head()

,Tempo,Estação,Movimentação,Original_473,Original_269,Zero,Maçã-Verde,Tangerina,Citrus,Açaí-Guaraná,Pêssego,TARGET
0,2018-01-01,Paraíso,70277.0,86.0,65.0,65.0,43.0,43.0,43.0,43.0,43.0,NORMAL
1,2018-01-02,Paraíso,73302.0,73.0,58.0,58.0,38.0,37.0,42.0,41.0,39.0,NORMAL
2,2018-01-03,Paraíso,71711.0,64.0,50.0,50.0,33.0,34.0,42.0,39.0,35.0,NORMAL
3,2018-01-04,Paraíso,71740.0,52.0,48.0,42.0,27.0,29.0,36.0,37.0,32.0,NORMAL
4,2018-01-05,Paraíso,71035.0,41.0,40.0,32.0,21.0,23.0,36.0,34.0,26.0,NORMAL


In [12]:
df_training_dataset_rmcolumns["Movimentação"] = df_training_dataset_rmcolumns["Movimentação"].astype(int)

df_training_dataset_rmcolumns["LAST_MOV"] = df_training_dataset_rmcolumns.groupby(['Estação'])['Movimentação'].shift()
df_training_dataset_rmcolumns["DIFF_LAST_MOV"] = df_training_dataset_rmcolumns.groupby(['Estação'])["LAST_MOV"].diff()

df_training_dataset_rmcolumns["MIN_PROD"] = df_training_dataset_rmcolumns[["Original_473", "Original_269", "Zero", "Maçã-Verde", "Tangerina", "Citrus", "Açaí-Guaraná", "Pêssego"]].min(axis = 1, skipna = True) 
df_training_dataset_rmcolumns["DIFF_MIN_PROD"] = df_training_dataset_rmcolumns[["MIN_PROD"]].diff().diff()

df_training_dataset_rmcolumns.tail()

,Tempo,Estação,Movimentação,Original_473,Original_269,Zero,Maçã-Verde,Tangerina,Citrus,Açaí-Guaraná,Pêssego,TARGET,LAST_MOV,DIFF_LAST_MOV,MIN_PROD,DIFF_MIN_PROD
17011,2019-12-26,Hospital São Paulo,25702,53.0,54.0,26.0,38.0,23.0,25.0,11.0,37.0,NORMAL,25250.0,-1117.0,11.0,-20.0
17012,2019-12-27,Hospital São Paulo,26236,43.0,52.0,17.0,33.0,18.0,21.0,5.0,35.0,REABASTECER,25702.0,452.0,5.0,0.0
17013,2019-12-28,Hospital São Paulo,27174,34.0,50.0,9.0,28.0,15.0,16.0,43.0,32.0,NORMAL,26236.0,534.0,9.0,10.0
17014,2019-12-29,Hospital São Paulo,26877,22.0,47.0,65.0,27.0,11.0,16.0,39.0,30.0,NORMAL,27174.0,938.0,11.0,-2.0
17015,2019-12-30,Hospital São Paulo,26759,9.0,42.0,56.0,22.0,8.0,10.0,35.0,26.0,NORMAL,26877.0,-297.0,8.0,-5.0


In [13]:
df_training_dataset_rmcolumns["Original_473"] = df_training_dataset_rmcolumns["Original_473"].astype(int)
df_training_dataset_rmcolumns["Original_269"] = df_training_dataset_rmcolumns["Original_269"].astype(int)
df_training_dataset_rmcolumns["Zero"] = df_training_dataset_rmcolumns["Zero"].astype(int)
df_training_dataset_rmcolumns["Maçã-Verde"] = df_training_dataset_rmcolumns["Maçã-Verde"].astype(int)
df_training_dataset_rmcolumns["Tangerina"] = df_training_dataset_rmcolumns["Tangerina"].astype(int)
df_training_dataset_rmcolumns["Citrus"] = df_training_dataset_rmcolumns["Citrus"].astype(int)
df_training_dataset_rmcolumns["Açaí-Guaraná"] = df_training_dataset_rmcolumns["Açaí-Guaraná"].astype(int)
df_training_dataset_rmcolumns["Pêssego"] = df_training_dataset_rmcolumns["Pêssego"].astype(int)

df_training_dataset_rmcolumns["Estação"] = df_training_dataset_rmcolumns["Estação"].astype("category")

In [14]:
df_training = df_training_dataset_rmcolumns.copy()
df_training.columns

Index(['Tempo', 'Estação', 'Movimentação', 'Original_473', 'Original_269',
       'Zero', 'Maçã-Verde', 'Tangerina', 'Citrus', 'Açaí-Guaraná', 'Pêssego',
       'TARGET', 'LAST_MOV', 'DIFF_LAST_MOV', 'MIN_PROD', 'DIFF_MIN_PROD'],
      dtype='object')

In [15]:
df_training.dropna(inplace=True)

In [16]:
features = df_training[
    [
        'Estação', 'Original_473', 'Original_269', 'Zero', 'Maçã-Verde', 'Tangerina',
       'Citrus', 'Açaí-Guaraná', 'Pêssego', 'DIFF_LAST_MOV', 'DIFF_MIN_PROD'
    ]
]
target = df_training['TARGET']  ## NÃO TROQUE O NOME DA VARIÁVEL TARGET.

In [17]:
features.dtypes

Estação          category
Original_473        int64
Original_269        int64
Zero                int64
Maçã-Verde          int64
Tangerina           int64
Citrus              int64
Açaí-Guaraná        int64
Pêssego             int64
DIFF_LAST_MOV     float64
DIFF_MIN_PROD     float64
dtype: object

### Dividindo nosso conjunto de dados em conjuntos de treinamento e teste

In [18]:
from sklearn import preprocessing

num_d = features.select_dtypes(exclude=['category'])

sc = preprocessing.RobustScaler().fit(num_d)
num_d.columns

Index(['Original_473', 'Original_269', 'Zero', 'Maçã-Verde', 'Tangerina',
       'Citrus', 'Açaí-Guaraná', 'Pêssego', 'DIFF_LAST_MOV', 'DIFF_MIN_PROD'],
      dtype='object')

In [19]:
features[num_d.columns] = sc.transform(num_d)

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/site-packages/pandas/core/indexing.py:1736: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value[:, i].tolist())


In [20]:
features.head()

,Estação,Original_473,Original_269,Zero,Maçã-Verde,Tangerina,Citrus,Açaí-Guaraná,Pêssego,DIFF_LAST_MOV,DIFF_MIN_PROD
2,Paraíso,0.395349,0.43750,0.424242,0.428571,0.454545,0.809524,0.681818,0.523810,1.300844,0.153846
3,Paraíso,0.116279,0.37500,0.181818,0.142857,0.227273,0.523810,0.590909,0.380952,-0.672438,-0.153846
4,Paraíso,-0.139535,0.12500,-0.121212,-0.142857,-0.045455,0.523810,0.454545,0.095238,0.020092,0.000000
5,Paraíso,-0.372093,-0.03125,-0.303030,-0.190476,-0.272727,0.476190,0.272727,-0.047619,-0.293684,0.230769
6,Paraíso,-0.534884,-0.21875,-0.515152,-0.476190,-0.363636,0.238095,0.090909,-0.190476,0.874212,-0.076923


In [21]:
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTENC
# from imblearn.over_sampling import SMOTE

X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.01, random_state=1008)

smc = SMOTENC(categorical_features=[0], random_state=1006)
# smc = SMOTE(random_state=1006)

X_train, y_train = smc.fit_resample(X_train, y_train)

### Treinando uma árvore de decisão

In [22]:
# Método para criar um árvore de decisão
from lightgbm import LGBMClassifier
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV

dtc = LGBMClassifier(learning_rate=0.001, n_estimators=10000, class_weight='balanced')

param_search = {'max_depth' : [-1], 
                'num_leaves': [32]
               }

tscv = TimeSeriesSplit(n_splits=2)
gsearch = GridSearchCV(estimator=dtc, cv=tscv,
                        param_grid=param_search)

# dtc.fit(X_train, y_train, categorical_feature=["Estação"])
# dtc.fit(X_train, y_train)
gsearch.fit(X_train, y_train, categorical_feature=["Estação"])

/usr/local/lib/python3.7/site-packages/lightgbm/basic.py:1551: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')


GridSearchCV(cv=TimeSeriesSplit(max_train_size=None, n_splits=2),
             estimator=LGBMClassifier(class_weight='balanced',
                                      learning_rate=0.001, n_estimators=10000),
             param_grid={'max_depth': [-1], 'num_leaves': [32]})

In [23]:
# Método para criar um árvore de decisão
from lightgbm import LGBMClassifier
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV

dtc = LGBMClassifier(learning_rate=0.001, n_estimators=10000, class_weight='balanced')

param_search = {'max_depth' : [-1, 3, 5], 
                'num_leaves': [32],
                'lambda_l1': [0, 1, 1.5],
                'boosting_type': ['gbdt', 'goss', 'dart']
               }

tscv = TimeSeriesSplit(n_splits=5)
gsearch = GridSearchCV(estimator=dtc, cv=tscv,
                        param_grid=param_search)

# dtc.fit(X_train, y_train, categorical_feature=["Estação"])
# dtc.fit(X_train, y_train)
gsearch.fit(X_train, y_train, categorical_feature=["Estação"])

/usr/local/lib/python3.7/site-packages/lightgbm/basic.py:1551: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')


KeyboardInterrupt: 

### Fazendo previsões na amostra de teste

In [26]:
# y_pred = dtc.predict(X_test)
y_pred = gsearch.predict(X_test)
print(y_pred)

GridSearchCV(cv=TimeSeriesSplit(max_train_size=None, n_splits=5),
             estimator=LGBMClassifier(class_weight='balanced',
                                      learning_rate=0.001, n_estimators=10000),
             param_grid={'boosting_type': ['gbdt', 'goss', 'dart'],
                         'lambda_l1': [0, 1, 1.5], 'max_depth': [-1, 3, 5],
                         'num_leaves': [32]})

In [ ]:
len(y_pred)

### Analisando a qualidade do modelo através da matriz de confusão

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import itertools


def plot_confusion_matrix(cm, target_names, title='Confusion matrix', cmap=None, normalize=True):
    accuracy = np.trace(cm) / float(np.sum(cm))
    misclass = 1 - accuracy
    if cmap is None:
        cmap = plt.get_cmap('Blues')
    plt.figure(figsize=(9, 9))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    if target_names is not None:
        tick_marks = np.arange(len(target_names))
        plt.xticks(tick_marks, target_names, rotation=45)
        plt.yticks(tick_marks, target_names)
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    thresh = cm.max() / 1.5 if normalize else cm.max() / 2
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        if normalize:
            plt.text(j, i, "{:0.2f}".format(cm[i, j]),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")
        else:
            plt.text(j, i, "{:,}".format(cm[i, j]),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label\naccuracy={:0.4f}; misclass={:0.4f}'.format(accuracy, misclass))
    plt.show()

In [ ]:
from sklearn.metrics import confusion_matrix


plot_confusion_matrix(confusion_matrix(y_test, y_pred), ['NORMAL', 'REABASTECER'])

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(y_test, y_pred))

Como entrega da sua solução, esperamos os resultados classificados no seguinte dataset chamado "to_be_scored.csv":

### Download da "folha de respostas"

In [ ]:
!wget --no-check-certificate --content-disposition https://gitlab.com/JoaoPedroPP/datasets/-/raw/master/ntn/to_be_scored.csv
df_to_be_scored = pd.read_csv(r'to_be_scored.csv')
df_to_be_scored.tail()

In [ ]:
#df_to_be_scored = pd.read_csv('to_be_scored.csv')
#df_to_be_scored.tail()

# Atenção!

O dataframe ``to_be_scored`` é a sua "folha de respostas". Note que a coluna "TARGET" não existe nessa amostra, que não pode ser então utilizada para treino de modelos de aprendizado supervisionado.

In [ ]:
df_to_be_scored.info()

<hr>

# Atenção!

# Para poder aplicar seu modelo e classificar a folha de respostas, você precisa primeiro aplicar as mesmas transformações com colunas que você aplicou no dataset de treino.

# Não remova ou adicione linhas na folha de respostas. 

# Não altere a ordem das linhas na folha de respostas.

# Ao final, as 1000 entradas devem estar classificadas, com os valores previstos em uma coluna chamada "target"

<hr>

Na célula abaixo, repetimos rapidamente os mesmos passos de pré-processamento usados no exemplo dado com árvore de decisão

In [ ]:
# 1 - Removendo linhas com valores NaN
df_to_be_scored_1 = df_to_be_scored.dropna(axis='index', how='any', subset=['Tempo', 'Estação', 'LAT', 'LONG', 'Movimentação', 'Original_473', 'Original_269', 'Zero', 'Maçã-Verde', 'Tangerina', 'Citrus', 'Açaí-Guaraná', 'Pêssego'])

# 2 - Inputando zeros nos valores faltantes
impute_zeros.fit(X=df_to_be_scored_1)
df_to_be_scored_2 = pd.DataFrame.from_records(
    data=impute_zeros.transform(
        X=df_to_be_scored_1
    ),
    columns=df_to_be_scored_1.columns
)

# 3 - Remoção de colunas
df_to_be_scored_3 = df_to_be_scored_2.drop(columns=['LAT', 'LONG'], inplace=False)

# 4 - Encoding com "dummy variables" (se necessário)
# df_to_be_scored_4 = pd.get_dummies(df_to_be_scored_3, columns=['Váriavel com dummy'])
# df_to_be_scored_4 = df_to_be_scored_3

df_to_be_scored_4 = df_to_be_scored_3.copy()

df_to_be_scored_4["Tempo"] = pd.to_datetime(df_to_be_scored_4["Tempo"], format='%Y-%m-%d', errors='coerce')
df_to_be_scored_4["Estação"] = df_to_be_scored_4["Estação"].astype("category")
df_to_be_scored_4["Movimentação"] = df_to_be_scored_4["Movimentação"].astype(int)
df_to_be_scored_4["LAST_MOV"] = df_to_be_scored_4.groupby(['Estação'])['Movimentação'].shift()
df_to_be_scored_4["DIFF_LAST_MOV"] = df_to_be_scored_4.groupby(['Estação'])["LAST_MOV"].diff()
df_to_be_scored_4['LAST_MOV'] = df_to_be_scored_4['LAST_MOV'].fillna(0)
df_to_be_scored_4['DIFF_LAST_MOV'] = df_to_be_scored_4['DIFF_LAST_MOV'].fillna(0)
df_to_be_scored_4.drop(['Tempo', 'Movimentação'], axis='columns', inplace=True)

df_to_be_scored_4["MIN_PROD"] = df_to_be_scored_4[["Original_473", "Original_269", "Zero", "Maçã-Verde", "Tangerina", "Citrus", "Açaí-Guaraná", "Pêssego"]].min(axis = 1, skipna = True) 
df_to_be_scored_4["DIFF_MIN_PROD"] = df_to_be_scored_4[["MIN_PROD"]].diff().diff()

num_d = df_to_be_scored_4.select_dtypes(exclude=['category'])
sc = preprocessing.RobustScaler().fit(num_d)
df_to_be_scored_4[num_d.columns] = sc.transform(num_d)
df_to_be_scored_4 = df_to_be_scored_4.drop(columns=['LAST_MOV', 'MIN_PROD'], inplace=False)

In [ ]:
df_to_be_scored_4.tail()

In [ ]:
df_to_be_scored_4.dtypes

<hr>

Pode ser verificado abaixo que as colunas da folha de resposta agora são idênticas às que foram usadas para treinar o modelo:

In [ ]:
df_training[
    [
          'Estação', 'Original_473', 'Original_269', 'Zero', 'Maçã-Verde', 'Tangerina',
       'Citrus', 'Açaí-Guaraná', 'Pêssego', 'DIFF_LAST_MOV', 'DIFF_MIN_PROD'
    ]
].columns

In [ ]:
df_to_be_scored_4.columns

# Atenção

Para todas colunas que não existirem no "df_to_be_scored", você pode usar a técnica abaixo para adicioná-las:

In [ ]:
y_pred = dtc.predict(df_to_be_scored_4)
df_to_be_scored_4['TARGET'] = y_pred
df_to_be_scored_4.tail()

### Salvando a folha de respostas como um arquivo .csv para ser submetido

In [ ]:
df_to_be_scored_4.to_csv("results.csv", index=False)

# Atenção

# A execução da célula acima irá criar um novo "data asset" no seu projeto no Watson Studio. Você precisará realizar o download deste arquivo juntamente com este notebook e criar um arquivo zip com os arquivos **results.csv** e **notebook.ipynb** para submissão. (os arquivos devem estar nomeados desta forma)

<hr>

## Parabéns!

Se você já está satisfeito com a sua solução, vá até a página abaixo e envie os arquivos necessários para submissão.

# https://tnt.maratona.dev
